In [1]:
#!pip install pandas==1.3.5

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px

import plotly.io as pio
pio.renderers.default = "iframe"

from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score

import pickle 

In [3]:
pd.__version__

'2.2.3'

# Steps

1. Understanding the Data
2. Preprocess + EDA + Feature Selection
3. Extract input and output cols
4. Scale the values
5. Train test split
6. Train the model
7. Evaluate the model/model selection
8. Tuning the model
9. Deploy the model

# Understanding the Data
### You need to answer some of the questions regarding your data:

In [5]:
df = pd.read_csv('collegePlace.csv')

### 1. How big is the data?

In [6]:
df.shape

(2966, 8)

### 2. How does the data look like?

In [7]:
df.head()
#df.tail()

Age  Gender                         Stream  Internships  CGPA  Hostel  \
0   22    Male  Electronics And Communication            1     8       1   
1   21  Female               Computer Science            0     7       1   
2   22  Female         Information Technology            1     6       0   
3   21    Male         Information Technology            0     8       0   
4   22    Male                     Mechanical            0     8       1   

   HistoryOfBacklogs  PlacedOrNot  
0                  1            1  
1                  1            1  
2                  0            1  
3                  1            1  
4                  0            1

In [ ]:
df.sample(4)

Age Gender            Stream  Internships  CGPA  Hostel  \
1757   23   Male  Computer Science            1     6       0   
2897   22   Male        Mechanical            0     8       1   
1625   21   Male             Civil            0     8       0   
628    22   Male             Civil            1     6       0   

      HistoryOfBacklogs  PlacedOrNot  
1757                  0            0  
2897                  0            1  
1625                  0            1  
628                   0            0

### 3. What is the data type of cols?

In [9]:
df.dtypes

Age                   int64
Gender               object
Stream               object
Internships           int64
CGPA                  int64
Hostel                int64
HistoryOfBacklogs     int64
PlacedOrNot           int64
dtype: object

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2966 entries, 0 to 2965
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Age                2966 non-null   int64 
 1   Gender             2966 non-null   object
 2   Stream             2966 non-null   object
 3   Internships        2966 non-null   int64 
 4   CGPA               2966 non-null   int64 
 5   Hostel             2966 non-null   int64 
 6   HistoryOfBacklogs  2966 non-null   int64 
 7   PlacedOrNot        2966 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 185.5+ KB


### 4. How does the data look mathematically?

In [13]:
df.describe()

Age  Internships         CGPA       Hostel  HistoryOfBacklogs  \
count  2966.000000  2966.000000  2966.000000  2966.000000        2966.000000   
mean     21.485840     0.703641     7.073837     0.269049           0.192178   
std       1.324933     0.740197     0.967748     0.443540           0.394079   
min      19.000000     0.000000     5.000000     0.000000           0.000000   
25%      21.000000     0.000000     6.000000     0.000000           0.000000   
50%      21.000000     1.000000     7.000000     0.000000           0.000000   
75%      22.000000     1.000000     8.000000     1.000000           0.000000   
max      30.000000     3.000000     9.000000     1.000000           1.000000   

       PlacedOrNot  
count  2966.000000  
mean      0.552596  
std       0.497310  
min       0.000000  
25%       0.000000  
50%       1.000000  
75%       1.000000  
max       1.000000

### 6. How is the correlation between features(other Colms) and target col?

In [56]:
df_test=df[['Age','Internships','CGPA','Hostel','HistoryOfBacklogs','PlacedOrNot']]

df_test.corr()['PlacedOrNot']

Age                  0.052444
Internships          0.164916
CGPA                 0.533497
Hostel              -0.048414
HistoryOfBacklogs   -0.055885
PlacedOrNot          1.000000
Name: PlacedOrNot, dtype: float64

# Preprocessing + EDA + Feature Selection

## Preprocessing

### Basic Preprocessing: 
### 1. Check if the dataframe has any null values, handle if present.
### 2. Check if there is any duplicate row in the datafame, handle if present.

### 1. Are there any missing values?

In [16]:
# missing values
df.isnull().sum()

#drop null values if present
#df.dropna(inplace=True)

Age                  0
Gender               0
Stream               0
Internships          0
CGPA                 0
Hostel               0
HistoryOfBacklogs    0
PlacedOrNot          0
dtype: int64

### Null Values Not Present

### 2. Are there duplicate values?

In [17]:
# duplicate rows
print(df.duplicated().sum())

#drop duplicates
df.drop_duplicates(inplace=True)

1829


In [18]:
# Check if the duplicate rows are removed
print(df.duplicated().sum())

0


## EDA

### 1. Plot the graph to visualize the output wrt 2 major features

In [ ]:
#Plot the graph to visualize the output wrt major features
#plt.scatter(df['CGPA'],df['Internships'],c=df['PlacedOrNot']) 
fig = px.scatter(df, x="CGPA", y="Internships", color="PlacedOrNot",
                 hover_data=['CGPA'])
fig.show()

### 2. Plot the count of placed and not placed

In [57]:
fig = px.histogram(df, x='PlacedOrNot', color='PlacedOrNot', barmode='group')
fig.show()

In [58]:
# Pie Chart
fig = px.pie(df, values=df['PlacedOrNot'].value_counts().values, names=df['PlacedOrNot'].value_counts().index, title='Placed Vs Not Placed')
fig.show()

ValueError: Length mismatch: Expected axis has 1137 elements, new values have 2 elements

### Get the Max and Min Age of the person who is placed

In [22]:
print("Max Age of Placed Person: ",df[(df['Age'] == df['Age'].max()) & (df['PlacedOrNot']==1)]['Age'].values[0])
print("Min Age of Placed Person: ",df[(df['Age'] == df['Age'].min()) & (df['PlacedOrNot']==1)]['Age'].values[0])

Max Age of Placed Person:  30
Min Age of Placed Person:  19


### Get the Max and Min Internships done by the student who is placed

In [23]:
print("Max Internships Done by the Placed Student: ",df[(df['Internships'] == df['Internships'].max()) & (df['PlacedOrNot']==1)]['Internships'].values[0])
print("No of students who did max Internships and are placed: ",df[(df['Internships'] == df['Internships'].max()) & (df['PlacedOrNot']==1)]['Internships'].value_counts().values[0])

print("Min Internships Done by the Placed Person: ",df[(df['Internships'] == df['Internships'].min()) & (df['PlacedOrNot']==1)]['Internships'].values[0])
print("No of students who did min Internships and are placed: ",df[(df['Internships'] == df['Internships'].min()) & (df['PlacedOrNot']==1)]['Internships'].value_counts().values[0])


Max Internships Done by the Placed Student:  3
No of students who did max Internships and are placed:  35
Min Internships Done by the Placed Person:  0
No of students who did min Internships and are placed:  232


### Get the Max and Min CGPA of the student who is placed

In [24]:
print("Max CGPA of Placed Student: ",df[(df['CGPA'] == df['CGPA'].max()) & (df['PlacedOrNot']==1)]['CGPA'].values[0])
print("No of students has max CGPA and are placed: ",df[(df['CGPA'] == df['CGPA'].max()) & (df['PlacedOrNot']==1)]['CGPA'].value_counts().values[0])

print("Min CGPA of Placed Person: ",df[(df['CGPA'] == df['CGPA'].min()) & (df['PlacedOrNot']==1)]['CGPA'].values[0])
print("No of students has min CGPA and are placed: ",df[(df['CGPA'] == df['CGPA'].min()) & (df['PlacedOrNot']==1)]['CGPA'].value_counts().values[0])


Max CGPA of Placed Student:  9
No of students has max CGPA and are placed:  99
Min CGPA of Placed Person:  5
No of students has min CGPA and are placed:  5


### Statistical representation 

In [25]:
fig = px.box(df, y='CGPA')
fig.show()

In [26]:
fig = px.box(df, y='Age')
fig.show()

In [27]:
fig = px.box(df, y=['Internships','CGPA', 'Age'])
fig.show()

## Convert Categorical Data to Numeric

In [28]:
# convert Gender column to numeric
df['Gender'] = df['Gender'].map({'Male': 1, 'Female': 0})

In [29]:
df['Stream'].unique()

array(['Electronics And Communication', 'Computer Science',
       'Information Technology', 'Mechanical', 'Electrical', 'Civil'],
      dtype=object)

In [30]:
# convert Stream column to numeric
df['Stream'] = df['Stream'].map({'Electronics And Communication': 1, 
                                 'Computer Science': 2,
                                'Information Technology': 3,
                                'Mechanical':4,
                                'Electrical':5,
                                'Civil':6})

In [31]:
df.sample(5)

Age  Gender  Stream  Internships  CGPA  Hostel  HistoryOfBacklogs  \
1716   23       1       2            1     5       0                  0   
79     21       1       2            1     8       0                  0   
2830   22       1       1            3     8       0                  1   
1112   21       1       2            2     9       0                  0   
1740   21       1       2            2     5       0                  0   

      PlacedOrNot  
1716            0  
79              1  
2830            1  
1112            1  
1740            0

In [ ]:
# # you can reduce the features using PCA
# pca = PCA(n_components=2)\\
# X_pca = pca.fit_transform(X)
# X_pca_transform = pd.DataFrame(data=X_pca)
# #Plot the graph 
# plt.scatter(X_pca_transform[0],X_pca_transform[1],c=y)

# Extract input and output cols

In [33]:
X = df.iloc[:,0:7]
y = df.iloc[:,-1]
X

Age  Gender  Stream  Internships  CGPA  Hostel  HistoryOfBacklogs
0      22       1       1            1     8       1                  1
1      21       0       2            0     7       1                  1
2      22       0       3            1     6       0                  0
3      21       1       3            0     8       0                  1
4      22       1       4            0     8       1                  0
...   ...     ...     ...          ...   ...     ...                ...
2946   23       1       3            1     7       1                  1
2952   23       1       4            0     8       1                  0
2954   23       0       2            1     8       0                  1
2958   23       1       2            0     6       0                  1
2960   23       1       4            1     7       1                  0

[1137 rows x 7 columns]

In [34]:
print(X.shape)
print(y.shape)

(1137, 7)
(1137,)


# Train test split

In [35]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33)

In [36]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(761, 7)
(376, 7)
(761,)
(376,)


# Scale the values

In [37]:
scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

# Train and Evaluate the model

## Logistic Regression

In [39]:
#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
#using Logistic Regression
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()

# Without Scaling 
clf.fit(X_train,y_train) 
y_pred = clf.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())

# With Scaling 
clf.fit(X_train_scale,y_train) 
y_pred = clf.predict(X_test_scale)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())

#scaling has not much effect

Without Scaling and CV:  0.7180851063829787
Without Scaling and With CV:  0.7292720437457281
With Scaling and Without CV:  0.7207446808510638
With Scaling and With CV:  0.7319036226930964


## SGD Classifier

In [40]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier
#Using SGD Classifier
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(max_iter=1000, tol=1e-3)

# Without Scaling
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())

# With Scaling 
clf.fit(X_train_scale,y_train) 
y_pred = clf.predict(X_test_scale)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())


Without Scaling and CV:  0.43617021276595747
Without Scaling and With CV:  0.6346206425153793
With Scaling and Without CV:  0.726063829787234
With Scaling and With CV:  0.7174470266575529


In [41]:
from sklearn.linear_model import Perceptron
# this is same as SGDClassifier(loss="perceptron", eta0=1, learning_rate="constant", penalty=None)

clf = Perceptron(tol=1e-3, random_state=0)
# Without Scaling
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())

# With Scaling 
clf.fit(X_train_scale,y_train) 
y_pred = clf.predict(X_test_scale)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())


Without Scaling and CV:  0.5691489361702128
Without Scaling and With CV:  0.6045967190704032
With Scaling and Without CV:  0.6675531914893617
With Scaling and With CV:  0.6570061517429938


In [42]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html#sklearn.linear_model.LogisticRegressionCV
# Using LogisticRegressionCV
from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV(cv=5, random_state=0)

# Without Scaling
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())

# With Scaling 
clf.fit(X_train_scale,y_train) 
y_pred = clf.predict(X_test_scale)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())

Without Scaling and CV:  0.7207446808510638


C:\Users\devdn\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



Without Scaling and With CV:  0.7187457279562544
With Scaling and Without CV:  0.7180851063829787
With Scaling and With CV:  0.7279391660970609


## Decision Tree Classifier

In [43]:
# https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
# Using DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)

#without scaling
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())

# With Scaling 
clf.fit(X_train_scale,y_train) 
y_pred = clf.predict(X_test_scale)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())

Without Scaling and CV:  0.7367021276595744
Without Scaling and With CV:  0.737183868762816
With Scaling and Without CV:  0.7367021276595744
With Scaling and With CV:  0.7345522898154477


## Random Forest Classifier

In [44]:
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=10, random_state=0)

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())

# With Scaling 
clf.fit(X_train_scale,y_train) 
y_pred = clf.predict(X_test_scale)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())

Without Scaling and CV:  0.7606382978723404
Without Scaling and With CV:  0.7753588516746411
With Scaling and Without CV:  0.7606382978723404
With Scaling and With CV:  0.7740430622009569


## Support Vector Machines

In [ ]:
#https://scikit-learn.org/stable/modules/svm.html

In [45]:
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC
from sklearn.svm import SVC
#clf = SVC(gamma='auto')

svc = SVC()
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
clf = GridSearchCV(svc, parameters)

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Best Parameters:", clf.best_params_)
print("Best Score:", clf.best_score_)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())

# With Scaling 
clf.fit(X_train_scale,y_train) 
y_pred = clf.predict(X_test_scale)
print("Best Parameters:", clf.best_params_)
print("Best Score:", clf.best_score_)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())

Best Parameters: {'C': 10, 'kernel': 'rbf'}
Best Score: 0.7490196078431373
Without Scaling and CV:  0.7313829787234043
Without Scaling and With CV:  0.743796992481203
Best Parameters: {'C': 1, 'kernel': 'rbf'}
Best Score: 0.7831699346405229
With Scaling and Without CV:  0.7659574468085106
With Scaling and With CV:  0.7753075871496924


In [46]:
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.NuSVC.html#sklearn.svm.NuSVC
from sklearn.svm import NuSVC
clf = NuSVC()

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())

# With Scaling 
clf.fit(X_train_scale,y_train) 
y_pred = clf.predict(X_test_scale)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())

Without Scaling and CV:  0.7712765957446809
Without Scaling and With CV:  0.760868079289132
With Scaling and Without CV:  0.773936170212766
With Scaling and With CV:  0.7753075871496924


In [47]:
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0, tol=1e-5)

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())

# With Scaling 
clf.fit(X_train_scale,y_train) 
y_pred = clf.predict(X_test_scale)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())

Without Scaling and CV:  0.7074468085106383
Without Scaling and With CV:  0.7332365003417636
With Scaling and Without CV:  0.7180851063829787
With Scaling and With CV:  0.7293062200956937


## Naive Bayes

In [48]:
#https://scikit-learn.org/stable/modules/naive_bayes.html

In [49]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

#y_pred = gnb.fit(X_train, y_train).predict(X_test)
#print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (y_test != y_pred).sum()))

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())

# With Scaling 
clf.fit(X_train_scale,y_train) 
y_pred = clf.predict(X_test_scale)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())

Without Scaling and CV:  0.7473404255319149
Without Scaling and With CV:  0.7476930963773069
With Scaling and Without CV:  0.7473404255319149
With Scaling and With CV:  0.7476930963773069


In [50]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())


Without Scaling and CV:  0.5957446808510638
Without Scaling and With CV:  0.6412679425837322


In [59]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())

# With Scaling 
clf.fit(X_train_scale,y_train) 
y_pred = clf.predict(X_test_scale)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())

Without Scaling and CV:  0.5478723404255319
Without Scaling and With CV:  0.5873718386876282
With Scaling and Without CV:  0.7553191489361702
With Scaling and With CV:  0.7569548872180452


In [60]:
from sklearn.naive_bayes import CategoricalNB
clf = CategoricalNB()

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
#scores = cross_val_score(clf, X_train, y_train, cv=5)
#print("Without Scaling and With CV: ",scores.mean())

Without Scaling and CV:  0.776595744680851


## K-Nearest Neighbors

In [61]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=3)

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without Scaling and CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("Without Scaling and With CV: ",scores.mean())


# With Scaling 
clf.fit(X_train_scale,y_train) 
y_pred = clf.predict(X_test_scale)
print("With Scaling and Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train_scale, y_train, cv=10)
print("With Scaling and With CV: ",scores.mean())

Without Scaling and CV:  0.7579787234042553
Without Scaling and With CV:  0.7397812713602188
With Scaling and Without CV:  0.723404255319149
With Scaling and With CV:  0.7030416951469584


# Model selection

### As we can see, among all the algorithm implemented Random Forest gives you max Accuracy.

In [62]:
clf = RandomForestClassifier(max_depth=10, random_state=0)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("With CV: ",scores.mean())
print("Precision Score: ", precision_score(y_test, y_pred))
print("Recall Score: ", recall_score(y_test, y_pred))
print("F1 Score: ", f1_score(y_test, y_pred))


Without CV:  0.7606382978723404
With CV:  0.7753588516746411
Precision Score:  0.8369565217391305
Recall Score:  0.719626168224299
F1 Score:  0.7738693467336684


# Tuning the model

#### Using Hyper-Parameter tuning using GridsearchCV Hypertune the parameters for Random forest and get best parameters

In [63]:
param_grid = {
    'bootstrap': [False,True],
    'max_depth': [5,8,10, 20],
    'max_features': [3, 4, 5, None],
    'min_samples_split': [2, 10, 12],
    'n_estimators': [100, 200, 300]
}

rfc = RandomForestClassifier()

clf = GridSearchCV(estimator = rfc, param_grid = param_grid, cv = 5, n_jobs = -1, verbose = 1)

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred))
print(clf.best_params_)
print(clf.best_estimator_)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits
Accuracy:  0.8138297872340425
{'bootstrap': True, 'max_depth': 5, 'max_features': None, 'min_samples_split': 12, 'n_estimators': 100}
RandomForestClassifier(max_depth=5, max_features=None, min_samples_split=12)


#### Training the final model with best parameters:

In [64]:
clf = RandomForestClassifier(bootstrap=False, max_depth=5,max_features=None,
                             min_samples_split=2,
                             n_estimators=100, random_state=0)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Without CV: ",accuracy_score(y_test,y_pred))
scores = cross_val_score(clf, X_train, y_train, cv=10)
print("With CV: ",scores.mean())
print("Precision Score: ", precision_score(y_test, y_pred))
print("Recall Score: ", recall_score(y_test, y_pred))
print("F1 Score: ", f1_score(y_test, y_pred))

Without CV:  0.8164893617021277
With CV:  0.8279049897470949
Precision Score:  0.9289940828402367
Recall Score:  0.7336448598130841
F1 Score:  0.8198433420365535


# Deploy the model

## Dump the model

In [65]:
pickle.dump(clf,open('model.pkl','wb'))

## Load the model for usage

In [66]:
model = pickle.load(open('model.pkl','rb'))

## Make Predictions

In [67]:
age = 22
gender = 1 # 1=Male, 0=Female
stream = 3  # Electronics And Communication': 1, 
#              'Computer Science': 2,
#              'Information Technology': 3,
#              'Mechanical':4,
#              'Electrical':5,
#              'Civil':6
Internships = 2
CGPA = 6
Hostel = 1 # 1= stay in hostel, 0=not staying in hostel
HistoryOfBacklogs = 0 # 1 = had backlogs, 0=no backlogs

prediction = clf.predict([[age,gender,stream,Internships,CGPA,Hostel,HistoryOfBacklogs]])
prediction

C:\Users\devdn\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but RandomForestClassifier was fitted with feature names



array([1])

# Try all the classifiers from SKLearn

### Get all the estimators in sklearn(Algorithms)

In [68]:
# Import all estimators list
from sklearn.utils import all_estimators


estimators = all_estimators(type_filter='classifier')
classification_estimators = []
i = 1
for name, class_ in estimators:
    classification_estimators.append(class_.__name__)
    print(f'{i}. {class_.__name__}')
    i += 1

1. AdaBoostClassifier
2. BaggingClassifier
3. BernoulliNB
4. CalibratedClassifierCV
5. CategoricalNB
6. ClassifierChain
7. ComplementNB
8. DecisionTreeClassifier
9. DummyClassifier
10. ExtraTreeClassifier
11. ExtraTreesClassifier
12. FixedThresholdClassifier
13. GaussianNB
14. GaussianProcessClassifier
15. GradientBoostingClassifier
16. HistGradientBoostingClassifier
17. KNeighborsClassifier
18. LabelPropagation
19. LabelSpreading
20. LinearDiscriminantAnalysis
21. LinearSVC
22. LogisticRegression
23. LogisticRegressionCV
24. MLPClassifier
25. MultiOutputClassifier
26. MultinomialNB
27. NearestCentroid
28. NuSVC
29. OneVsOneClassifier
30. OneVsRestClassifier
31. OutputCodeClassifier
32. PassiveAggressiveClassifier
33. Perceptron
34. QuadraticDiscriminantAnalysis
35. RadiusNeighborsClassifier
36. RandomForestClassifier
37. RidgeClassifier
38. RidgeClassifierCV
39. SGDClassifier
40. SVC
41. SelfTrainingClassifier
42. StackingClassifier
43. TunedThresholdClassifierCV
44. VotingClassifier


In [69]:
for name, class_ in estimators:
    module_name = str(class_).split("'")[1].split(".")[1]
    class_name = class_.__name__
    print(f'from sklearn.{module_name} import {class_name}')

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import CategoricalNB
from sklearn.multioutput import ClassifierChain
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import FixedThresholdClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.discriminant_analysis import Li

In [71]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import CategoricalNB
from sklearn.multioutput import ClassifierChain
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import NearestCentroid
from sklearn.svm import NuSVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

### Train the model using all the algorithms and compare accuracy metrics

### 1. Without Scaling

In [72]:

for estimator in classification_estimators:
    try:
        print("ALgorithm: ", estimator)
        clf = eval(estimator+'()')
        clf.fit(X_train,y_train)
        y_pred = clf.predict(X_test)
        print("Without CV: ",accuracy_score(y_test,y_pred))
        scores = cross_val_score(clf, X_train, y_train, cv=10)
        print("With CV: ",scores.mean())
        print("Precision Score: ", precision_score(y_test, y_pred))
        print("Recall Score: ", recall_score(y_test, y_pred))
        print("F1 Score: ", f1_score(y_test, y_pred))
        print("____________________________________________________")
    except:
        print("Error")
    


ALgorithm:  AdaBoostClassifier
Without CV:  0.7978723404255319
With CV:  0.7911141490088859
Precision Score:  0.9207317073170732
Recall Score:  0.705607476635514
F1 Score:  0.798941798941799
____________________________________________________
ALgorithm:  BaggingClassifier
Without CV:  0.7340425531914894
With CV:  0.7437969924812029
Precision Score:  0.7794117647058824
Recall Score:  0.7429906542056075
F1 Score:  0.7607655502392344
____________________________________________________
ALgorithm:  BernoulliNB
Without CV:  0.5478723404255319
With CV:  0.5873718386876282
Precision Score:  0.5714285714285714
Recall Score:  0.822429906542056
F1 Score:  0.6743295019157088
____________________________________________________
ALgorithm:  CalibratedClassifierCV
Without CV:  0.7101063829787234
With CV:  0.7332365003417636
Precision Score:  0.7397260273972602
Recall Score:  0.7570093457943925
F1 Score:  0.7482678983833718
____________________________________________________
ALgorithm:  Categorical

### 2. With Scaling

In [73]:
#for estimator in classification_estimators:
#    try:
#        print("ALgorithm: ", estimator)
#        clf = eval(estimator+'()')
#        clf.fit(X_train_scale,y_train)
#        y_pred = clf.predict(X_test_scale)
#        print("Without CV: ",accuracy_score(y_test,y_pred))
#        scores = cross_val_score(clf, X_train_scale, y_train, cv=5)
#        print("With CV: ",scores.mean())
#        print("Precision Score: ", precision_score(y_test, y_pred))
#        print("Recall Score: ", recall_score(y_test, y_pred))
#        print("F1 Score: ", f1_score(y_test, y_pred))
#        print("____________________________________________________")
#    except:
#        print("Error")

## With the above analysis, we have found that we get highest accuracy with GradientBoostingClassifier. using this we now tune the parameters to get best parameters.

In [74]:
#parameters = {
##    "loss":["deviance"],
#    "learning_rate": [0.05, 0.1, 0.15, 0.2],
#    "min_samples_split": np.linspace(0.1, 0.5, 12),
#    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
#    "max_depth":[8,10,12],
#    "max_features":["log2","sqrt"],
#    "criterion": ["friedman_mse"],
#    "subsample":[0.5,0.85, 0.9, 1.0],
#    "n_estimators":[10, 50, 80]
#    }

#clf = GridSearchCV(GradientBoostingClassifier(), parameters, cv=10, n_jobs=-1, verbose = 1)
#clf.fit(X_train,y_train)
#y_pred = clf.predict(X_test)
##print("Accuracy: ",accuracy_score(y_test,y_pred))
#print(clf.best_params_)
#print(clf.best_estimator_)

# Train using best parameters

In [75]:
#clf = GradientBoostingClassifier(learning_rate=0.2, max_depth=8, max_features='log2',
#                           min_samples_leaf=0.1, min_samples_split=0.1,
#                           n_estimators=50)
#clf.fit(X_train,y_train)
#y_pred = clf.predict(X_test)
#print("Without CV: ",accuracy_score(y_test,y_pred))
#scores = cross_val_score(clf, X_train, y_train, cv=10)
#print("With CV: ",scores.mean())
#print("Precision Score: ", precision_score(y_test, y_pred))
#print("Recall Score: ", recall_score(y_test, y_pred))
#print("F1 Score: ", f1_score(y_test, y_pred))

### Thank you